# LanceDB - a vector database for LLM application

In [2]:
import lancedb

db = lancedb.connect(uri = "vector_database")
db

LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database')

# Create a table 

In [3]:
db

LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database')

In [6]:
import json 

with open("data/animal_text_embeddings.json", "r") as file:
    data = json.loads(file.read())

In [7]:
table_animals = db.create_table("animals_text", exist_ok=True, data = data)
table_animals

LanceTable(name='animals_text', version=1, _conn=LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database'))

In [8]:
table_animals = db.create_table("animals_text", data = data, mode = "overwrite")
table_animals

LanceTable(name='animals_text', version=2, _conn=LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database'))

In [9]:
table_animals.head()

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa.","A large gray elephant drinking water.","A fast cheetah sprinting across the savannah.","A colorful parrot perched on a branch."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1],[0.88,0.22,0.55],[0.95,0.12,0.72],[0.25,0.66,0.81]]]

In [10]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"


In [11]:
more_data = [
  {
    "text": "A small brown dog running.",
    "vector": [0.12, 0.85, 0.33]
  },

  {
    "text": "A cat resting quietly on a sofa.",
    "vector": [0.4, 0.91, 0.1]
  },
  
  ]

table_animals.add(more_data)

AddResult(version=3)

In [12]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A small brown dog running.,"[0.12, 0.85, 0.33]"
7,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"


## Create an empty table and then delete it 

In [13]:
from lancedb.pydantic import LanceModel

class JokeSchema(LanceModel):
    joke: str
    rating: int

db.create_table(name = "jokes", schema = JokeSchema, exist_ok=True)
db

LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database')

In [14]:
db.table_names()

['animals_text', 'jokes']

In [15]:
db.drop_table("jokes")

In [16]:
db.table_names

<bound method LanceDBConnection.table_names of LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database')>

# Open existing table

In [17]:
db.open_table("animals_text").head(2)

pyarrow.Table
text: string
vector: fixed_size_list<item: float>[3]
  child 0, item: float
----
text: [["A small brown dog running.","A cat resting quietly on a sofa."]]
vector: [[[0.12,0.85,0.33],[0.4,0.91,0.1]]]

## Vector search in LanceDB

In [18]:
table_animals.to_pandas()

,text,vector
0,A small brown dog running.,"[0.12, 0.85, 0.33]"
1,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"
2,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]"
3,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]"
4,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]"
5,A frog sitting on a lily pad.,"[0.14, 0.44, 0.27]"
6,A small brown dog running.,"[0.12, 0.85, 0.33]"
7,A cat resting quietly on a sofa.,"[0.4, 0.91, 0.1]"


In [19]:
query_vector = [0.5, 0.2, 0.9]

table_animals.search(query_vector).limit(3).to_pandas()

,text,vector,_distance
0,A fast cheetah sprinting across the savannah.,"[0.95, 0.12, 0.72]",0.2413
1,A large gray elephant drinking water.,"[0.88, 0.22, 0.55]",0.2673
2,A colorful parrot perched on a branch.,"[0.25, 0.66, 0.81]",0.2822


# Embedding API

In [21]:
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

model = get_registry().get("gemini-text").create(name = "gemini-embedding-001")
model

GeminiText(max_retries=7, name='gemini-embedding-001', query_task_type='retrieval_query', source_task_type='retrieval_document')

In [22]:
model.generate_embeddings("hello")

[[-0.010894743,
  -0.0028489418,
  0.00893425,
  -0.054541513,
  -0.007433747,
  0.017504206,
  0.014653749,
  0.02498823,
  0.01153613,
  -0.004031619,
  0.0019689437,
  -0.015896015,
  -0.0033653777,
  0.015975969,
  0.09177606,
  -0.02413529,
  0.026621163,
  -0.015998762,
  -0.009193494,
  -0.0032317825,
  0.017314918,
  -0.014104707,
  -0.000599844,
  -0.013850878,
  0.025816025,
  0.027195023,
  0.017755596,
  0.009800108,
  0.022374708,
  0.017374959,
  -0.0027653421,
  0.0067342324,
  0.00040601502,
  0.002778903,
  -0.004352005,
  0.011505135,
  0.022893922,
  -0.010107965,
  -0.0130854435,
  0.022383824,
  0.00030385726,
  -0.007134147,
  0.002239337,
  -0.0012981425,
  7.041567e-05,
  0.011082349,
  -0.005637078,
  -0.025727957,
  0.0059076254,
  0.0042295936,
  0.0017543633,
  0.011937581,
  -0.020759232,
  -0.14057352,
  -0.011109075,
  0.006916541,
  -0.0025381926,
  0.021703891,
  0.0076077846,
  -0.029725924,
  0.010393887,
  0.026465422,
  -0.014564507,
  -0.008960989,

In [23]:
import numpy as np

helo_embedding = np.array(model.generate_embeddings("hello"))
helo_embedding.shape


(5, 3072)

In [25]:
class JokeModel(LanceModel):
    joke: str = model.SourceField()
    vector: Vector(3072) = model.VectorField()

table_jokes = db.create_table("jokes", schema=JokeModel, exist_ok=True)
table_jokes

LanceTable(name='jokes', version=1, _conn=LanceDBConnection(uri='/Users/abdulrahmanfahmi/Documents/Github/Dataplattformar_maskininl-rning_artificiell-intelligens_DE24_Abdulrahman_fahmi/vector_database'))

In [26]:
import pandas as pd 

with open("data/jokes.json", "r") as file:
    jokes_data = json.loads(file.read())

pd.DataFrame(jokes_data)

,jokes
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.
5,I asked the data lake if it had my file. It sa...
6,My math teacher said I’m average… how mean!
7,Python is so friendly even whitespace gets a say.
8,The C# compiler walked into a bar. The bartend...
9,Never trust an atom—they make up everything.


In [28]:
import pandas as pd 

with open("data/jokes.json", "r") as file:
    jokes_data = json.loads(file.read())

df_jokes = pd.DataFrame(jokes_data).rename({"jokes": "joke"}, axis=1)
df_jokes

,joke
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.
5,I asked the data lake if it had my file. It sa...
6,My math teacher said I’m average… how mean!
7,Python is so friendly even whitespace gets a say.
8,The C# compiler walked into a bar. The bartend...
9,Never trust an atom—they make up everything.


In [29]:
df_jokes.head()

,joke
0,Parallel lines have so much in common—it’s sad...
1,"ETL stands for “Extract, Transform, Leave for ..."
2,What do you call a snake that runs your script...
3,"Gold walks into a bar. The bartender says, “Au..."
4,C# devs don’t argue; they just throw exceptions.


## add data to table

In [30]:
table_jokes.head()

pyarrow.Table
joke: string not null
vector: fixed_size_list<item: float>[3072]
  child 0, item: float
----
joke: []
vector: []

In [31]:
table_jokes.add(df_jokes)

AddResult(version=2)

In [32]:
table_jokes.head()

pyarrow.Table
joke: string not null
vector: fixed_size_list<item: float>[3072]
  child 0, item: float
----
joke: [["Parallel lines have so much in common—it’s sad they’ll never meet.","ETL stands for “Extract, Transform, Leave for the next person.”","What do you call a snake that runs your scripts? A py-thon (ton of errors).","Gold walks into a bar. The bartender says, “Au, get out!”","C# devs don’t argue; they just throw exceptions."]]
vector: [[[-0.024001757,0.01247358,-0.024144737,-0.06704516,0.017059995,...,0.011402721,-0.017770408,0.011606138,0.0004488597,0.012175956],[-0.015356114,0.0211365,-0.021389864,-0.07957475,0.008829045,...,0.023671096,0.00070166244,0.003873497,0.0006493248,0.01587487],[-0.01761013,0.0031474787,-0.015632002,-0.060952485,0.003696307,...,0.015392491,-0.00025712984,0.0064440705,0.0017164479,-0.00869067],[-0.024867292,0.013314825,-0.016261652,-0.07383461,0.0085606,...,0.031690735,-0.0027016324,0.017647061,0.008275158,-0.0067860777],[-0.0068662865,-0.005415149,

In [33]:
table_jokes.to_pandas()

,joke,vector
0,Parallel lines have so much in common—it’s sad...,"[-0.024001757, 0.01247358, -0.024144737, -0.06..."
1,"ETL stands for “Extract, Transform, Leave for ...","[-0.015356114, 0.0211365, -0.021389864, -0.079..."
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0..."
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0..."
4,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0..."
5,I asked the data lake if it had my file. It sa...,"[-0.026169129, 0.01796462, -0.013938847, -0.09..."
6,My math teacher said I’m average… how mean!,"[-0.004950074, 0.016501933, -0.01651097, -0.07..."
7,Python is so friendly even whitespace gets a say.,"[-0.025112161, 0.008470194, -0.023098752, -0.0..."
8,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07..."
9,Never trust an atom—they make up everything.,"[-0.020950116, 0.00022863908, -0.015684012, -0..."


# Perform vector search

In [34]:
table_jokes.search("data engineer jokes").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.470778
1,"Data engineer motto: If it works, don’t touch ...","[-0.020296954, 0.020327171, -0.009069326, -0.0...",0.556300
2,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.673100
3,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.688158
4,I tried to explain async/await to my friend… n...,"[-0.016174542, 0.012602222, -0.015254588, -0.0...",0.694943
5,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.695609
6,I asked the data lake if it had my file. It sa...,"[-0.026169129, 0.01796462, -0.013938847, -0.09...",0.701310
7,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.708513


In [35]:
table_jokes.search("Give me some jokes on C#").limit(8).to_pandas()

,joke,vector,_distance
0,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.464983
1,Why did the C# developer go broke? He kept usi...,"[-0.02175711, 0.0073709465, -0.015515881, -0.0...",0.498013
2,Why is C# like a musical? It has so many classes.,"[-0.012926053, 0.0034431089, -0.017093537, -0....",0.526406
3,C# devs don’t argue; they just throw exceptions.,"[-0.0068662865, -0.005415149, 0.0044965413, -0...",0.620939
4,C# programmers love coffee—they're always work...,"[-0.0079965545, 0.0151333995, -0.009256328, -0...",0.633884
5,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.638709
6,I tried to explain async/await to my friend… n...,"[-0.016174542, 0.012602222, -0.015254588, -0.0...",0.648802
7,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.650193


In [36]:
table_jokes.search("Give me some jokes on nature").limit(8).to_pandas()

,joke,vector,_distance
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.587673
1,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.687205
2,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.708071
3,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.713952
4,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.762457
5,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.764604
6,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.769536
7,Why’s 6 afraid of 7? Because 7 8 9.,"[-0.040820926, 0.0074244644, -0.02972158, -0.0...",0.791893


## Hybrid search

In [37]:
table_jokes.create_fts_index("joke", replace=True)

In [44]:
from lancedb import rerankers

reranker = rerankers.RRFReranker()

result = table_jokes.search(
    "give me nature related jokes",
    query_type="hybrid",
    vector_column_name="vector",
    fts_columns="joke"

).rerank(reranker=reranker).limit(8).to_pandas()

result

,joke,vector,_relevance_score
0,Why do data engineers hate nature? Too many un...,"[-0.027916763, 0.0047387416, -0.018934403, -0....",0.032522
1,I told a chemistry joke… there was no reaction.,"[-0.022922393, 0.017959604, -0.029222224, -0.0...",0.032018
2,"Gold walks into a bar. The bartender says, “Au...","[-0.024867292, 0.013314825, -0.016261652, -0.0...",0.016129
3,Why did the chemist ground his kids? Because t...,"[-0.023257235, 0.016145445, -0.029016329, -0.0...",0.015873
4,The C# compiler walked into a bar. The bartend...,"[-0.01868987, 0.018796643, -0.009748903, -0.07...",0.015385
5,What do you call a snake that runs your script...,"[-0.01761013, 0.0031474787, -0.015632002, -0.0...",0.015152
6,Why did the Python programmer get bitten? Beca...,"[-0.021944793, 0.0030636177, -0.019837778, -0....",0.014925
7,Why’s 6 afraid of 7? Because 7 8 9.,"[-0.040820926, 0.0074244644, -0.02972158, -0.0...",0.014706


## Rule of thumb